# Neural Network from Scratch - Research Scientist Interview Prep

**Author:** Rekhi  
**Purpose:** Demonstrate deep understanding of neural network fundamentals  

---

## What This Notebook Covers

1. **Foundations**: Activation functions and their derivatives
2. **Architecture**: Weight initialization strategies
3. **Forward Propagation**: How data flows through the network
4. **Backpropagation**: Computing gradients via chain rule
5. **Gradient Checking**: Verifying backprop correctness
6. **Training Loop**: Gradient descent optimization
7. **Experiments**: Testing on different data distributions

---

## Part 1: Imports and Configuration

In [ ]:
import numpy as np
import matplotlib.pyplot as plt

# Reproducibility
RANDOM_SEED = 42
np.random.seed(RANDOM_SEED)

# Plot settings
plt.style.use('seaborn-v0_8-whitegrid')
plt.rcParams['figure.figsize'] = (10, 6)

---

## Part 2: Activation Functions

### Why Activation Functions Matter

Without activation functions, a neural network is just a linear transformation:  
`y = W2(W1*x + b1) + b2 = W_combined*x + b_combined`

Activation functions introduce **non-linearity**, allowing networks to learn complex patterns.

---

### ReLU (Rectified Linear Unit)

**Formula:** `f(x) = max(0, x)`  
**Derivative:** `f'(x) = 1 if x > 0 else 0`

**Why ReLU:**
- Computationally efficient (just a threshold)
- Reduces vanishing gradient problem (gradient is 1 for positive values)
- Sparse activation (neurons can be "off")

**Limitation:** "Dying ReLU" - neurons can get stuck at 0

In [ ]:
def relu(z):
    """
    ReLU activation: f(z) = max(0, z)
    
    Args:
        z: numpy array, pre-activation values
    
    Returns:
        numpy array, activated values
    """
    return np.maximum(0, z)


def relu_derivative(z):
    """
    Derivative of ReLU: f'(z) = 1 if z > 0 else 0
    
    Args:
        z: numpy array, pre-activation values
    
    Returns:
        numpy array, gradients
    """
    return (z > 0).astype(float)

### Sigmoid

**Formula:** `f(x) = 1 / (1 + e^(-x))`  
**Derivative:** `f'(x) = f(x) * (1 - f(x))`

**Why Sigmoid:**
- Outputs in range (0, 1) - good for probabilities
- Smooth gradient

**Limitations:**
- Vanishing gradient (gradient near 0 for large |x|)
- Outputs not zero-centered
- Computationally expensive (exponential)

In [ ]:
def sigmoid(z):
    """
    Sigmoid activation: f(z) = 1 / (1 + e^(-z))
    
    Args:
        z: numpy array, pre-activation values
    
    Returns:
        numpy array, activated values in (0, 1)
    """
    # Clip to prevent overflow in exp
    z_clipped = np.clip(z, -500, 500)
    return 1 / (1 + np.exp(-z_clipped))


def sigmoid_derivative(z):
    """
    Derivative of sigmoid: f'(z) = f(z) * (1 - f(z))
    
    Args:
        z: numpy array, pre-activation values
    
    Returns:
        numpy array, gradients
    """
    s = sigmoid(z)
    return s * (1 - s)

### Tanh (Hyperbolic Tangent)

**Formula:** `f(x) = (e^x - e^(-x)) / (e^x + e^(-x))`  
**Derivative:** `f'(x) = 1 - f(x)^2`

**Why Tanh:**
- Zero-centered outputs (-1, 1)
- Stronger gradients than sigmoid

**Limitation:** Still has vanishing gradient for large |x|

In [ ]:
def tanh(z):
    """
    Tanh activation: f(z) = (e^z - e^(-z)) / (e^z + e^(-z))
    
    Args:
        z: numpy array, pre-activation values
    
    Returns:
        numpy array, activated values in (-1, 1)
    """
    return np.tanh(z)


def tanh_derivative(z):
    """
    Derivative of tanh: f'(z) = 1 - f(z)^2
    
    Args:
        z: numpy array, pre-activation values
    
    Returns:
        numpy array, gradients
    """
    t = tanh(z)
    return 1 - t ** 2

### Visualize Activation Functions

In [ ]:
z_values = np.linspace(-5, 5, 200)

fig, axes = plt.subplots(2, 3, figsize=(14, 8))

# Row 1: Activation functions
axes[0, 0].plot(z_values, relu(z_values), 'b-', linewidth=2)
axes[0, 0].set_title('ReLU')
axes[0, 0].set_xlabel('z')
axes[0, 0].set_ylabel('f(z)')
axes[0, 0].axhline(y=0, color='k', linestyle='--', alpha=0.3)
axes[0, 0].axvline(x=0, color='k', linestyle='--', alpha=0.3)

axes[0, 1].plot(z_values, sigmoid(z_values), 'g-', linewidth=2)
axes[0, 1].set_title('Sigmoid')
axes[0, 1].set_xlabel('z')
axes[0, 1].set_ylabel('f(z)')
axes[0, 1].axhline(y=0.5, color='k', linestyle='--', alpha=0.3)
axes[0, 1].axvline(x=0, color='k', linestyle='--', alpha=0.3)

axes[0, 2].plot(z_values, tanh(z_values), 'r-', linewidth=2)
axes[0, 2].set_title('Tanh')
axes[0, 2].set_xlabel('z')
axes[0, 2].set_ylabel('f(z)')
axes[0, 2].axhline(y=0, color='k', linestyle='--', alpha=0.3)
axes[0, 2].axvline(x=0, color='k', linestyle='--', alpha=0.3)

# Row 2: Derivatives
axes[1, 0].plot(z_values, relu_derivative(z_values), 'b-', linewidth=2)
axes[1, 0].set_title('ReLU Derivative')
axes[1, 0].set_xlabel('z')
axes[1, 0].set_ylabel("f'(z)")

axes[1, 1].plot(z_values, sigmoid_derivative(z_values), 'g-', linewidth=2)
axes[1, 1].set_title('Sigmoid Derivative')
axes[1, 1].set_xlabel('z')
axes[1, 1].set_ylabel("f'(z)")

axes[1, 2].plot(z_values, tanh_derivative(z_values), 'r-', linewidth=2)
axes[1, 2].set_title('Tanh Derivative')
axes[1, 2].set_xlabel('z')
axes[1, 2].set_ylabel("f'(z)")

plt.tight_layout()
plt.show()

print("Key Observations:")
print("- ReLU derivative is constant (1) for positive values - no vanishing gradient")
print("- Sigmoid derivative max is 0.25 at z=0 - gradients shrink quickly")
print("- Tanh derivative max is 1.0 at z=0 - better than sigmoid but still vanishes")

---

## Part 3: Loss Functions

### Mean Squared Error (MSE) - For Regression

**Formula:** `L = (1/n) * sum((y_true - y_pred)^2)`  
**Derivative w.r.t y_pred:** `dL/dy_pred = (2/n) * (y_pred - y_true)`

**Why MSE:**
- Penalizes large errors more (squared term)
- Differentiable everywhere
- Convex for linear models

In [ ]:
def mean_squared_error(y_true, y_pred):
    """
    Mean Squared Error loss.
    
    Args:
        y_true: numpy array, shape (n_samples, n_outputs), ground truth
        y_pred: numpy array, shape (n_samples, n_outputs), predictions
    
    Returns:
        float, mean squared error
    """
    n_samples = y_true.shape[0]
    return np.mean((y_true - y_pred) ** 2)


def mean_squared_error_derivative(y_true, y_pred):
    """
    Derivative of MSE w.r.t predictions.
    
    Args:
        y_true: numpy array, shape (n_samples, n_outputs)
        y_pred: numpy array, shape (n_samples, n_outputs)
    
    Returns:
        numpy array, gradient of loss w.r.t y_pred
    """
    n_samples = y_true.shape[0]
    return (2.0 / n_samples) * (y_pred - y_true)

---

## Part 4: Weight Initialization

### Why Initialization Matters

Bad initialization leads to:
- **Vanishing gradients**: Weights too small, signals shrink layer by layer
- **Exploding gradients**: Weights too large, signals blow up
- **Symmetry breaking**: All neurons learn the same thing

---

### He Initialization (for ReLU)

**Formula:** `W ~ N(0, sqrt(2/n_in))`

**Why:** ReLU kills half the values (negative), so we need larger variance to compensate.

---

### Xavier/Glorot Initialization (for Sigmoid/Tanh)

**Formula:** `W ~ N(0, sqrt(2/(n_in + n_out)))`

**Why:** Keeps variance stable across layers for symmetric activations.

In [ ]:
def initialize_parameters(layer_dimensions, initialization='he', seed=42):
    """
    Initialize weights and biases for all layers.
    
    Args:
        layer_dimensions: list, [n_input, n_hidden1, n_hidden2, ..., n_output]
        initialization: str, 'he' for ReLU, 'xavier' for sigmoid/tanh
        seed: int, random seed for reproducibility
    
    Returns:
        dict, containing W1, b1, W2, b2, ...
    
    Example:
        params = initialize_parameters([2, 4, 4, 1])
        # Creates: W1(2,4), b1(1,4), W2(4,4), b2(1,4), W3(4,1), b3(1,1)
    """
    np.random.seed(seed)
    parameters = {}
    num_layers = len(layer_dimensions)
    
    for layer_idx in range(1, num_layers):
        n_in = layer_dimensions[layer_idx - 1]   # neurons in previous layer
        n_out = layer_dimensions[layer_idx]      # neurons in current layer
        
        # Choose initialization based on activation
        if initialization == 'he':
            # He initialization: sqrt(2/n_in)
            std = np.sqrt(2.0 / n_in)
        elif initialization == 'xavier':
            # Xavier initialization: sqrt(2/(n_in + n_out))
            std = np.sqrt(2.0 / (n_in + n_out))
        else:
            # Simple small random
            std = 0.01
        
        # Initialize weights with appropriate scale
        parameters[f'W{layer_idx}'] = np.random.randn(n_in, n_out) * std
        
        # Initialize biases to zero
        parameters[f'b{layer_idx}'] = np.zeros((1, n_out))
    
    return parameters


# Test initialization
test_params = initialize_parameters([1, 10, 10, 1], initialization='he')
print("Network Architecture: [1, 10, 10, 1]")
print("\nParameter shapes:")
for key, value in test_params.items():
    print(f"  {key}: {value.shape}")

---

## Part 5: Forward Propagation

### The Math

For each layer `l`:

1. **Linear transformation:** `Z[l] = A[l-1] @ W[l] + b[l]`
2. **Activation:** `A[l] = activation(Z[l])`

Where:
- `A[0] = X` (input)
- `@` is matrix multiplication
- Last layer typically has no activation (for regression) or softmax (for classification)

### Why Cache?

We store `Z` and `A` values because backpropagation needs them to compute gradients.

In [ ]:
def forward_propagation(X, parameters, activation='relu'):
    """
    Perform forward pass through the network.
    
    Args:
        X: numpy array, shape (n_samples, n_features), input data
        parameters: dict, containing W1, b1, W2, b2, ...
        activation: str, 'relu', 'sigmoid', or 'tanh'
    
    Returns:
        y_pred: numpy array, shape (n_samples, n_outputs), predictions
        cache: dict, containing Z and A for each layer (needed for backprop)
    """
    cache = {'A0': X}  # Store input as A0
    A_current = X
    num_layers = len(parameters) // 2  # Each layer has W and b
    
    # Forward through hidden layers (with activation)
    for layer_idx in range(1, num_layers):
        # Get weights and biases for this layer
        W = parameters[f'W{layer_idx}']
        b = parameters[f'b{layer_idx}']
        
        # Linear transformation: Z = A_prev @ W + b
        Z = A_current @ W + b
        
        # Apply activation function
        if activation == 'relu':
            A_current = relu(Z)
        elif activation == 'sigmoid':
            A_current = sigmoid(Z)
        elif activation == 'tanh':
            A_current = tanh(Z)
        
        # Cache for backpropagation
        cache[f'Z{layer_idx}'] = Z
        cache[f'A{layer_idx}'] = A_current
    
    # Output layer (linear - no activation for regression)
    W_out = parameters[f'W{num_layers}']
    b_out = parameters[f'b{num_layers}']
    Z_out = A_current @ W_out + b_out
    y_pred = Z_out  # Linear output for regression
    
    cache[f'Z{num_layers}'] = Z_out
    cache[f'A{num_layers}'] = y_pred
    
    return y_pred, cache


# Test forward propagation
X_test = np.array([[0.5], [1.0], [1.5]])
y_pred_test, cache_test = forward_propagation(X_test, test_params)
print("Forward propagation test:")
print(f"  Input shape: {X_test.shape}")
print(f"  Output shape: {y_pred_test.shape}")
print(f"  Cache keys: {list(cache_test.keys())}")

---

## Part 6: Backpropagation

### The Chain Rule

To update `W[l]`, we need `dL/dW[l]`.

By chain rule:
```
dL/dW[l] = dL/dZ[l] * dZ[l]/dW[l]
         = dZ[l] * A[l-1].T
```

And for the next layer back:
```
dL/dA[l-1] = W[l].T @ dZ[l]
dL/dZ[l-1] = dL/dA[l-1] * activation_derivative(Z[l-1])
```

### Key Insight

We propagate `dZ` backwards through the network. At each layer:
1. Use `dZ` to compute `dW` and `db`
2. Compute `dA_prev` to pass to the previous layer
3. Apply activation derivative to get `dZ` for previous layer

In [ ]:
def backward_propagation(parameters, cache, y_true, activation='relu'):
    """
    Perform backward pass to compute gradients.
    
    Args:
        parameters: dict, containing W1, b1, W2, b2, ...
        cache: dict, containing Z and A from forward pass
        y_true: numpy array, shape (n_samples, n_outputs), ground truth
        activation: str, activation function used in hidden layers
    
    Returns:
        gradients: dict, containing dW1, db1, dW2, db2, ...
    """
    gradients = {}
    num_layers = len(parameters) // 2
    n_samples = y_true.shape[0]
    
    # Get prediction from cache
    y_pred = cache[f'A{num_layers}']
    
    # ========== OUTPUT LAYER ==========
    # dL/dZ_out = dL/dy_pred (since output is linear)
    # For MSE: dL/dy_pred = (2/n) * (y_pred - y_true)
    dZ_current = mean_squared_error_derivative(y_true, y_pred)
    
    # dW = A_prev.T @ dZ
    A_prev = cache[f'A{num_layers - 1}']
    gradients[f'dW{num_layers}'] = A_prev.T @ dZ_current
    
    # db = sum(dZ, axis=0)
    gradients[f'db{num_layers}'] = np.sum(dZ_current, axis=0, keepdims=True)
    
    # ========== HIDDEN LAYERS (going backwards) ==========
    # Propagate gradient to previous layer
    dA_prev = dZ_current @ parameters[f'W{num_layers}'].T
    
    for layer_idx in reversed(range(1, num_layers)):
        # Get cached values
        Z_current = cache[f'Z{layer_idx}']
        A_prev = cache[f'A{layer_idx - 1}']
        
        # Apply activation derivative: dZ = dA * activation'(Z)
        if activation == 'relu':
            dZ_current = dA_prev * relu_derivative(Z_current)
        elif activation == 'sigmoid':
            dZ_current = dA_prev * sigmoid_derivative(Z_current)
        elif activation == 'tanh':
            dZ_current = dA_prev * tanh_derivative(Z_current)
        
        # Compute gradients for this layer
        gradients[f'dW{layer_idx}'] = A_prev.T @ dZ_current
        gradients[f'db{layer_idx}'] = np.sum(dZ_current, axis=0, keepdims=True)
        
        # Propagate to previous layer (if not at first hidden layer)
        if layer_idx > 1:
            dA_prev = dZ_current @ parameters[f'W{layer_idx}'].T
    
    return gradients


# Test backpropagation
y_test = np.array([[0.1], [0.5], [0.9]])
grads_test = backward_propagation(test_params, cache_test, y_test)
print("Backpropagation test:")
print("  Gradient shapes:")
for key, value in grads_test.items():
    print(f"    {key}: {value.shape}")

---

## Part 7: Gradient Checking (IMPORTANT FOR INTERVIEWS)

### Why Gradient Checking?

Backprop is error-prone. Gradient checking verifies correctness by comparing:
- **Analytical gradient** (from backprop)
- **Numerical gradient** (from finite differences)

### The Method

For each parameter `theta`:
```
numerical_grad = (L(theta + epsilon) - L(theta - epsilon)) / (2 * epsilon)
```

### Relative Error

```
relative_error = |analytical - numerical| / (|analytical| + |numerical| + epsilon)
```

- `< 1e-7`: Excellent
- `< 1e-5`: Acceptable
- `> 1e-3`: Bug in backprop

In [ ]:
def gradient_check(parameters, X, y_true, activation='relu', epsilon=1e-7):
    """
    Verify backpropagation correctness using numerical gradients.
    
    Args:
        parameters: dict, network parameters
        X: numpy array, input data
        y_true: numpy array, ground truth
        activation: str, activation function
        epsilon: float, small value for finite differences
    
    Returns:
        float, maximum relative error across all parameters
    """
    # Get analytical gradients from backprop
    y_pred, cache = forward_propagation(X, parameters, activation)
    analytical_grads = backward_propagation(parameters, cache, y_true, activation)
    
    max_relative_error = 0
    
    # Check each parameter
    for param_name in parameters:
        param = parameters[param_name]
        grad_name = 'd' + param_name  # dW1, db1, etc.
        analytical_grad = analytical_grads[grad_name]
        
        # Compute numerical gradient for each element
        numerical_grad = np.zeros_like(param)
        
        # Iterate over each element in the parameter
        it = np.nditer(param, flags=['multi_index'], op_flags=['readwrite'])
        while not it.finished:
            idx = it.multi_index
            original_value = param[idx]
            
            # L(theta + epsilon)
            param[idx] = original_value + epsilon
            y_plus, _ = forward_propagation(X, parameters, activation)
            loss_plus = mean_squared_error(y_true, y_plus)
            
            # L(theta - epsilon)
            param[idx] = original_value - epsilon
            y_minus, _ = forward_propagation(X, parameters, activation)
            loss_minus = mean_squared_error(y_true, y_minus)
            
            # Numerical gradient
            numerical_grad[idx] = (loss_plus - loss_minus) / (2 * epsilon)
            
            # Restore original value
            param[idx] = original_value
            it.iternext()
        
        # Compute relative error
        numerator = np.abs(analytical_grad - numerical_grad)
        denominator = np.abs(analytical_grad) + np.abs(numerical_grad) + epsilon
        relative_error = np.max(numerator / denominator)
        
        max_relative_error = max(max_relative_error, relative_error)
        
        print(f"  {param_name}: relative error = {relative_error:.2e}")
    
    return max_relative_error


# Run gradient check on small network
print("Gradient Check Results:")
print("=" * 40)
small_params = initialize_parameters([1, 3, 3, 1], initialization='he')
X_check = np.array([[0.5], [1.0]])
y_check = np.array([[0.3], [0.7]])

max_error = gradient_check(small_params, X_check, y_check, activation='relu')
print("=" * 40)

if max_error < 1e-5:
    print(f"PASSED: Max error {max_error:.2e} < 1e-5")
else:
    print(f"WARNING: Max error {max_error:.2e} >= 1e-5 - check backprop!")

---

## Part 8: Parameter Update (Gradient Descent)

### Vanilla Gradient Descent

**Update rule:** `theta = theta - learning_rate * gradient`

### Learning Rate Considerations

- **Too large**: Overshoots minimum, may diverge
- **Too small**: Slow convergence
- **Just right**: Smooth decrease in loss

In [ ]:
def update_parameters(parameters, gradients, learning_rate):
    """
    Update parameters using gradient descent.
    
    Args:
        parameters: dict, current parameters
        gradients: dict, gradients from backprop
        learning_rate: float, step size
    
    Returns:
        dict, updated parameters
    """
    num_layers = len(parameters) // 2
    
    for layer_idx in range(1, num_layers + 1):
        parameters[f'W{layer_idx}'] -= learning_rate * gradients[f'dW{layer_idx}']
        parameters[f'b{layer_idx}'] -= learning_rate * gradients[f'db{layer_idx}']
    
    return parameters

---

## Part 9: Complete Training Pipeline

### The Training Loop

```
for each epoch:
    1. Forward pass: compute predictions
    2. Compute loss
    3. Backward pass: compute gradients
    4. Update parameters
```

In [ ]:
def train_network(X_train, y_train, layer_dimensions, 
                  learning_rate=0.01, num_epochs=1000,
                  activation='relu', initialization='he',
                  print_every=200, seed=42):
    """
    Train neural network using gradient descent.
    
    Args:
        X_train: numpy array, shape (n_samples, n_features)
        y_train: numpy array, shape (n_samples, n_outputs)
        layer_dimensions: list, [n_input, n_hidden1, ..., n_output]
        learning_rate: float, gradient descent step size
        num_epochs: int, number of training iterations
        activation: str, activation function for hidden layers
        initialization: str, weight initialization method
        print_every: int, print loss every N epochs
        seed: int, random seed
    
    Returns:
        parameters: dict, trained parameters
        loss_history: list, loss at each epoch
    """
    # Initialize parameters
    parameters = initialize_parameters(layer_dimensions, initialization, seed)
    loss_history = []
    
    print(f"Training network: {layer_dimensions}")
    print(f"Activation: {activation}, LR: {learning_rate}, Epochs: {num_epochs}")
    print("-" * 50)
    
    for epoch in range(num_epochs):
        # Forward propagation
        y_pred, cache = forward_propagation(X_train, parameters, activation)
        
        # Compute loss
        loss = mean_squared_error(y_train, y_pred)
        loss_history.append(loss)
        
        # Backward propagation
        gradients = backward_propagation(parameters, cache, y_train, activation)
        
        # Update parameters
        parameters = update_parameters(parameters, gradients, learning_rate)
        
        # Print progress
        if (epoch + 1) % print_every == 0 or epoch == 0:
            print(f"Epoch {epoch + 1:5d} | Loss: {loss:.6f}")
    
    print("-" * 50)
    print(f"Final Loss: {loss_history[-1]:.6f}")
    
    return parameters, loss_history

---

## Part 10: Experiments on Different Data

We test on three types of data to understand network behavior:

1. **Sinusoidal**: Non-linear, periodic - tests non-linear function approximation
2. **Linear**: Simple relationship - network should easily fit
3. **Polynomial**: Non-linear, non-periodic - tests smooth curve fitting

### Generate Datasets

In [ ]:
def generate_datasets(n_samples=200, noise_level=0.2, seed=42):
    """
    Generate three types of regression datasets.
    
    Returns:
        dict with 'sinusoidal', 'linear', 'polynomial' datasets
    """
    np.random.seed(seed)
    X = np.linspace(-2, 2, n_samples).reshape(-1, 1)
    noise = noise_level * np.random.randn(n_samples, 1)
    
    datasets = {
        'sinusoidal': {
            'X': X,
            'y': np.sin(3 * X) + noise,
            'description': 'y = sin(3x) + noise'
        },
        'linear': {
            'X': X,
            'y': 2.5 * X + 1.0 + noise,
            'description': 'y = 2.5x + 1 + noise'
        },
        'polynomial': {
            'X': X,
            'y': 1.5 * X**2 + 0.5 * X + 1.0 + noise,
            'description': 'y = 1.5x^2 + 0.5x + 1 + noise'
        }
    }
    
    return datasets


# Generate datasets
datasets = generate_datasets()

# Visualize datasets
fig, axes = plt.subplots(1, 3, figsize=(14, 4))

for idx, (name, data) in enumerate(datasets.items()):
    axes[idx].scatter(data['X'], data['y'], alpha=0.6, s=15)
    axes[idx].set_title(f"{name.capitalize()}\n{data['description']}")
    axes[idx].set_xlabel('X')
    axes[idx].set_ylabel('y')
    axes[idx].grid(True, alpha=0.3)

plt.tight_layout()
plt.show()

### Train on All Datasets

In [ ]:
# Training configuration
NETWORK_ARCHITECTURE = [1, 10, 10, 1]  # Input -> Hidden -> Hidden -> Output
LEARNING_RATE = 0.1
NUM_EPOCHS = 3000
ACTIVATION = 'relu'

# Store results
results = {}

for name, data in datasets.items():
    print(f"\n{'='*60}")
    print(f"Training on {name.upper()} data")
    print(f"{'='*60}")
    
    trained_params, loss_history = train_network(
        X_train=data['X'],
        y_train=data['y'],
        layer_dimensions=NETWORK_ARCHITECTURE,
        learning_rate=LEARNING_RATE,
        num_epochs=NUM_EPOCHS,
        activation=ACTIVATION,
        print_every=500
    )
    
    results[name] = {
        'parameters': trained_params,
        'loss_history': loss_history
    }

### Visualize Training Progress

In [ ]:
fig, axes = plt.subplots(1, 3, figsize=(14, 4))
colors = {'sinusoidal': 'blue', 'linear': 'green', 'polynomial': 'red'}

for idx, (name, result) in enumerate(results.items()):
    axes[idx].plot(result['loss_history'], color=colors[name], linewidth=1)
    axes[idx].set_title(f'{name.capitalize()} - Loss Curve')
    axes[idx].set_xlabel('Epoch')
    axes[idx].set_ylabel('MSE Loss')
    axes[idx].set_yscale('log')  # Log scale to see convergence better
    axes[idx].grid(True, alpha=0.3)
    
    # Annotate final loss
    final_loss = result['loss_history'][-1]
    axes[idx].annotate(f'Final: {final_loss:.4f}', 
                       xy=(0.95, 0.95), xycoords='axes fraction',
                       ha='right', va='top', fontsize=10,
                       bbox=dict(boxstyle='round', facecolor='wheat'))

plt.tight_layout()
plt.show()

### Visualize Predictions vs Ground Truth

In [ ]:
fig, axes = plt.subplots(1, 3, figsize=(14, 4))

for idx, (name, data) in enumerate(datasets.items()):
    X = data['X']
    y_true = data['y']
    params = results[name]['parameters']
    
    # Get predictions
    y_pred, _ = forward_propagation(X, params, activation=ACTIVATION)
    
    # Sort for smooth line plot
    sort_idx = np.argsort(X[:, 0])
    X_sorted = X[sort_idx]
    y_pred_sorted = y_pred[sort_idx]
    
    # Plot
    axes[idx].scatter(X, y_true, alpha=0.5, s=15, label='True Data', color='blue')
    axes[idx].plot(X_sorted, y_pred_sorted, color='red', linewidth=2, label='Prediction')
    axes[idx].set_title(f'{name.capitalize()}')
    axes[idx].set_xlabel('X')
    axes[idx].set_ylabel('y')
    axes[idx].legend()
    axes[idx].grid(True, alpha=0.3)

plt.tight_layout()
plt.show()

---

## Part 11: Analysis and Key Insights

### Convergence Analysis

In [ ]:
print("Convergence Analysis")
print("=" * 60)

for name, result in results.items():
    loss_history = result['loss_history']
    initial_loss = loss_history[0]
    final_loss = loss_history[-1]
    improvement = (initial_loss - final_loss) / initial_loss * 100
    
    # Find epoch where loss dropped below threshold
    threshold = final_loss * 1.1  # Within 10% of final
    convergence_epoch = next((i for i, loss in enumerate(loss_history) if loss < threshold), NUM_EPOCHS)
    
    print(f"\n{name.upper()}:")
    print(f"  Initial Loss: {initial_loss:.6f}")
    print(f"  Final Loss:   {final_loss:.6f}")
    print(f"  Improvement:  {improvement:.2f}%")
    print(f"  Converged at: ~epoch {convergence_epoch}")

### Key Takeaways

1. **Linear data converges fastest** - The simplest function to approximate

2. **Sinusoidal needs more capacity** - High frequency requires network to learn sharp turns

3. **Polynomial is intermediate** - Smooth curve, easier than sine but harder than line

4. **ReLU creates piecewise linear approximations** - Look at how the red line has "kinks"

---

## Part 12: Summary - What to Remember

### Core Equations

**Forward Pass:**
```
Z[l] = A[l-1] @ W[l] + b[l]
A[l] = activation(Z[l])
```

**Backward Pass (Chain Rule):**
```
dZ[L] = dL/dy_pred (output layer)
dW[l] = A[l-1].T @ dZ[l]
db[l] = sum(dZ[l], axis=0)
dA[l-1] = dZ[l] @ W[l].T
dZ[l-1] = dA[l-1] * activation'(Z[l-1])
```

**Update:**
```
W = W - lr * dW
b = b - lr * db
```

### Interview Points

1. **Why cache Z and A?** - Needed for backprop gradient computation

2. **Why He init for ReLU?** - ReLU kills half the values, need larger variance

3. **Why gradient checking?** - Verify backprop correctness (compare analytical vs numerical)

4. **Vanishing gradient?** - Sigmoid/tanh derivatives shrink, ReLU solves with constant gradient

5. **Why no activation on output?** - Regression needs unbounded output

6. **Matrix dimension rule:** `(n, m) @ (m, p) = (n, p)`

---

## Quick Reference Card

```
ACTIVATION FUNCTIONS
--------------------
ReLU:     f(z) = max(0, z)           f'(z) = 1 if z > 0 else 0
Sigmoid:  f(z) = 1/(1+e^-z)          f'(z) = f(z)(1-f(z))
Tanh:     f(z) = (e^z-e^-z)/(e^z+e^-z)   f'(z) = 1-f(z)^2

LOSS FUNCTIONS
--------------
MSE:      L = (1/n) * sum((y-y_hat)^2)    dL/dy_hat = (2/n)(y_hat-y)

INITIALIZATION
--------------
He (ReLU):     W ~ N(0, sqrt(2/n_in))
Xavier:        W ~ N(0, sqrt(2/(n_in+n_out)))

GRADIENT CHECKING
-----------------
numerical_grad = (L(theta+eps) - L(theta-eps)) / (2*eps)
relative_error = |analytical - numerical| / (|analytical| + |numerical|)
Good if < 1e-5
```